In [ ]:
import numpy as np
import re

import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk import sent_tokenize

from nltk.util import bigrams
from nltk.lm.preprocessing import padded_everygram_pipeline

import requests

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# you will need to leverage the requests package
r = requests.get(r'https://www.gutenberg.org/cache/epub/64317/pg64317.txt')
great_gatsby = r.text

# first, remove unwanted new line and tab characters from the text
for char in ["\n", "\r", "\d", "\t"]:
    great_gatsby = great_gatsby.replace(char, " ")

# check
print(great_gatsby[:100])
# remove the metadata at the beginning - this is slightly different for each book
great_gatsby = great_gatsby[983:]

﻿The Project Gutenberg eBook of The Great Gatsby        This ebook is for the use of anyone anywhere


In [ ]:
# this is simplified for demonstration
def sample_clean_text(text: str):
    # lowercase
    text = text.lower()

    # remove punctuation from text
    text = re.sub(r"[^\w\s]", "", text)

    # tokenize the text
    tokens = nltk.word_tokenize(text)

    # return your tokens
    return tokens

# call the function
sample_tokens = sample_clean_text(text = great_gatsby)

# check
print(sample_tokens[:50])
# create bigrams from the sample tokens
my_bigrams = bigrams(sample_tokens)

# check
list(my_bigrams)[:10]
# 2 is for bigrams
n = 2
#specify the text you want to use
text = great_gatsby
# step 1: tokenize the text into sentences
sentences = nltk.sent_tokenize(text)

# step 2: tokenize each sentence into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# step 3: convert each word to lowercase
tokenized_text = [[word.lower() for word in sent] for sent in tokenized_sentences]

#notice the sentence breaks and what the first 10 items of the tokenized text
print(tokenized_text[0])
# notice what the first 10 items are of the vocabulary
print(text[:10])

['then', 'wear', 'the', 'gold', 'hat', 'if', 'that', 'will', 'move', 'her', 'if', 'you', 'can', 'bounce', 'high', 'bounce', 'for', 'her', 'too', 'till', 'she', 'cry', 'lover', 'goldhatted', 'highbouncing', 'lover', 'i', 'must', 'have', 'you', 'thomas', 'parke', 'dinvilliers', 'i', 'in', 'my', 'younger', 'and', 'more', 'vulnerable', 'years', 'my', 'father', 'gave', 'me', 'some', 'advice', 'that', 'ive', 'been']
['then', 'wear', 'the', 'gold', 'hat', ',', 'if', 'that', 'will', 'move', 'her', ';', 'if', 'you', 'can', 'bounce', 'high', ',', 'bounce', 'for', 'her', 'too', ',', 'till', 'she', 'cry', '“', 'lover', ',', 'gold-hatted', ',', 'high-bouncing', 'lover', ',', 'i', 'must', 'have', 'you', '!', '”', 'thomas', 'parke', 'd', '’', 'invilliers', 'i', 'in', 'my', 'younger', 'and', 'more', 'vulnerable', 'years', 'my', 'father', 'gave', 'me', 'some', 'advice', 'that', 'i', '’', 've', 'been', 'turning', 'over', 'in', 'my', 'mind', 'ever', 'since', '.']
 Then wear


In [ ]:
# we imported this function from nltk
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)
from nltk.lm import MLE
# we imported this function from nltk linear models (lm)
# it is for Maximum Likelihood Estimation

# MLE is the model we will use
lm = MLE(n)
# currently the vocab length is 0: it has no prior knowledge
len(lm.vocab)
# fit the model
# training data is the bigrams and unigrams
# the vocab is all the sentence tokens in the corpus

lm.fit(train_data, padded_sents)
len(lm.vocab)

6953

In [ ]:
# inspect the model's vocabulary.
# be sure that a sentence you know exists (from tokenized_text) is in the
print(lm.vocab.lookup(tokenized_text[0]))


('then', 'wear', 'the', 'gold', 'hat', ',', 'if', 'that', 'will', 'move', 'her', ';', 'if', 'you', 'can', 'bounce', 'high', ',', 'bounce', 'for', 'her', 'too', ',', 'till', 'she', 'cry', '“', 'lover', ',', 'gold-hatted', ',', 'high-bouncing', 'lover', ',', 'i', 'must', 'have', 'you', '!', '”', 'thomas', 'parke', 'd', '’', 'invilliers', 'i', 'in', 'my', 'younger', 'and', 'more', 'vulnerable', 'years', 'my', 'father', 'gave', 'me', 'some', 'advice', 'that', 'i', '’', 've', 'been', 'turning', 'over', 'in', 'my', 'mind', 'ever', 'since', '.')


In [ ]:
# see what happens when we include a word that is not in the vocab.
print(lm.vocab.lookup('then wear the gold hat iphone .'.split()))

('then', 'wear', 'the', 'gold', 'hat', '<UNK>', '.')


In [ ]:

# how many times does daisy appear in the model?
print(lm.counts['daisy'])

# what is the probability of daisy appearing?
# this is technically the relative frequency of daisy appearing
lm.score('daisy')

183


0.0026549057726065954

In [ ]:
# how often does (daisy, and) occur and what is the relative frequency?
print(lm.counts[['daisy']]['and'])
lm.score('and', 'daisy'.split())

14


0.07650273224043716

In [ ]:
# what is the score of 'UNK'?

lm.score("<UNK>")

0.0

In [ ]:
# generate a 20 word sentence starting with the word, 'daisy'

print(lm.generate(20, text_seed= 'daisy', random_seed=42))

['other', ',', 'but', 'he', 'turned', 'to', 'their', 'cars', 'blocking', 'the', 'dull', 'light', ',', 'and', 'separated', 'only', 'building', 'in', 'the', 'adventitious']


In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(lm, num_words, text_seed, random_seed=42):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []
    for token in lm.generate(num_words, text_seed=text_seed, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

In [ ]:
# Now generate sentences that look much nicer.
generate_sent(lm, 20, text_seed='daisy', random_seed = 42)

'other, but he turned to their cars blocking the dull light, and separated only building in the adventitious'